# Checking the availablitiy of GPU

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:

print("Version of Tensorflow: ", tf.__version__)


Version of Tensorflow:  2.10.1


In [3]:
devices = tf.config.list_physical_devices('GPU')

In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
shakespeare_url = "https://homl.info/shakespeare"                                                         # webpage for text 
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

print(shakespeare_text[:80])



First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [6]:
#encoding of text
text_vec_layer = tf.keras.layers.TextVectorization(split="character",standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]


In [7]:
encoded -= 2                                            # dropping token 0 fro pad and 1 for unkown
n_tokens = text_vec_layer.vocabulary_size() - 2         # subtracting 2 from distinct chars
dataset_size = len(encoded)                             # total number of chars

In [8]:
dataset_size

1115394

In [9]:
# function that creat window like 1 window takes "hell" another take "ello" for word hello, if shuffle
def to_dataset(sequence, length, shuffle=False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)                                # create a tf dataset from the sequence
    ds = ds.window(length+1, shift=1, drop_remainder= True)                          # create overlapping window of length
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:                                                                      # shuffle the dataset 
        ds = ds.shuffle(buffer_size=100_000, seed=seed)                               
    ds = ds.batch(batch_size)                                                        # batches of given size         # map window where first element last item is left and second element first item is le                                                                          # prefetch for efficient training
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)


In [10]:
length = 100                                   # length of each sequence window
tf.random.set_seed(42)                           
train_set = to_dataset(encoded[:1_000_000], length = length, shuffle= True, seed=42)    # takes first 1,000,000 element
valid_set = to_dataset(encoded[1_00_000:1_060_000],length = length)                     # 1,000,000 to 1,060,000 element as validation
test_set = to_dataset(encoded[1_060_000:], length=length)                               # after 1,060,000 for test set

In [11]:
#@ Building and training char RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim =n_tokens, output_dim=16),                                                    # embedding layer encode charcter ids 
    tf.keras.layers.GRU(128, return_sequences=True),                                                           
    tf.keras.layers.Dense(n_tokens, activation="softmax")                                                             # give the probability for each character
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])                        # optimizing model
model_ckpt = tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model", monitor="val_accuracy", save_best_only=True)  # checkpoint to save best model with best validation acccuracy
history = model.fit(train_set, validation_data = valid_set,epochs=5,callbacks=[model_ckpt])                          # model train and save best model by callaback function

Epoch 1/5
  31244/Unknown - 389s 12ms/step - loss: 1.4077 - accuracy: 0.5692

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 549s 17ms/step - loss: 1.4077 - accuracy: 0.5692 - val_loss: 1.5104 - val_accuracy: 0.5432
Epoch 2/5
31245/31247 [============================>.] - ETA: 0s - loss: 1.3055 - accuracy: 0.5932

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 566s 18ms/step - loss: 1.2719 - accuracy: 0.6016 - val_loss: 1.4354 - val_accuracy: 0.5616
Epoch 5/5
31245/31247 [============================>.] - ETA: 0s - loss: 1.2652 - accuracy: 0.6029

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 556s 18ms/step - loss: 1.2652 - accuracy: 0.6029 - val_loss: 1.4306 - val_accuracy: 0.5623


In [12]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2),        # no padding and no unkown values
    model
])

In [21]:
y_proba = shakespeare_model.predict(["to be beautiful or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)                                          # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 10]

1/1 [==============================] - 0s 31ms/step


'n'

In [24]:
# function that takes the text as input and predict the next word temperature define logits
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]


In [25]:

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [30]:
extend_text('love is')

1/1 [==============================] - 0s 30ms/step


'love is as i saw,\nor we have speaks,--and indeed that i s'